# Construction of FFNN based on normalized training space
Upon stabilization and convergence of the model training loss curve to a non small number, normalization becomes imperative to homogenize the training space. Here, the training procedure is reiterated on a normalized training space employing two methods, namely min-max and Z-score normalization. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import save_model
from pathlib import Path

making home directory if it doesn't exist

In [ ]:
home_dir = Path().home()
ml_data = home_dir / 'ml_data'
if not ml_data.exists():
    ml_data.mkdir()

## Loading the training space and normalizing 


Loading data which is already generated:

In [ ]:
Design_space = np.load(ml_data / 'Design_space2.npy')
moment_capacity = np.load(ml_data / 'Mu2.npy')

Normalization of input data for training a neural network is a crucial preprocessing step that ensures stable and efficient training. It involves scaling the input features to a similar range, typically between 0 and 1 or with zero mean and unit variance. This helps prevent certain features from dominating the learning process and can accelerate convergence during training. Here's how we can normalize input data for training the neural network using min-max normalization and Z-score normalization:
1. Min-Max Normalization: Min-max normalization scales the input features to a fixed range, typically between 0 and 1.
$$
x_{\text {norm }}=\frac{x-\min (x)}{\max (x)-\min (x)}
$$


In [ ]:
# Min-Max Normalization
Design_space_normalized_min_max = (Design_space - np.min(Design_space, axis=0)) / (np.max(Design_space, axis=0) - np.min(Design_space, axis=0))
moment_capacity_normalized_min_max = (moment_capacity - np.min(moment_capacity, axis=0)) / (np.max(moment_capacity, axis=0) - np.min(moment_capacity, axis=0))

2. Z-score Normalization: Z-score normalization (also known as standardization) scales the input features to have a mean of 0 and a standard deviation of 1 .
$$
x_{\text {norm }}=\frac{x-\operatorname{mean}(x)}{\operatorname{std}(x)}
$$

In [ ]:
# Z-score Normalization:
Design_space_normalized_z_score = (Design_space - np.mean(Design_space, axis=0)) / np.std(Design_space, axis=0)
moment_capacity_normalized_z_score = (moment_capacity - np.mean(moment_capacity, axis=0)) / np.std(moment_capacity, axis=0)

## Visualization of the admissible design space for defined moment capacity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(25, 10))
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')
ax1.set_xlabel('Width (mm)')
ax1.set_ylabel('Height (mm)')
ax1.set_zlabel('Moment Capacity')
ax1.set_title('Reinforcement Ratio Surface')
ax2.set_xlabel('Width (mm)')
ax2.set_ylabel('Height (mm)')
ax2.set_zlabel('Moment Capacity')
ax2.set_title('Reinforcement Ratio Surface')

def plot_points_within_tolerance(Design_space_normalized_min_max, moment_capacity_normalized_min_max, target_z_value, tolerance, step_reinforcement, ax1, ax2):
    index=0
    for i in np.unique(Design_space_normalized_min_max[:, 0])[::step_reinforcement]:
        subset = Design_space_normalized_min_max[Design_space_normalized_min_max[:, 0] == i]
        z_values = np.array(moment_capacity_normalized_min_max[Design_space_normalized_min_max[:, 0] == i])
        for target_z_value in target_z_values:
            within_tolerance = np.isclose(z_values, target_z_value, rtol=tolerance)
            colors = ['red' if val else 'blue' for val in within_tolerance]
            red_indices = np.where(within_tolerance)[0]  # Get indices of red points
            # ax1 plots whole design space including inadmissible design space in blue and admissible in red
            ax1.scatter(subset[:, 1], subset[:, 2], z_values, label=f"Set {int(i)}", color=colors, alpha=0.1, s=5)
            # ax2 plots only the admissible design space linked by defined moment capacity in red
            ax2.scatter(subset[red_indices, 1], subset[red_indices, 2], z_values[red_indices], label=f"Set {int(i)}", color='red', alpha=0.1, s=1)
        # index += 1
        # if index == 50:
        #     break
target_z_values = [0.1, 0.25, 0.5, 0.8]
tolerance = 0.01
plot_step_for_reinforcement_ratio = 1


plot_points_within_tolerance(Design_space_normalized_min_max, moment_capacity_normalized_min_max, target_z_values, tolerance, plot_step_for_reinforcement_ratio, ax1, ax2)

plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

X = Design_space_normalized_min_max
y = moment_capacity_normalized_min_max


# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential([
    Dense(64, activation='relu', input_shape=(X.shape[1],)),
    Dense(64, activation='relu'),  # First hidden layer
    Dense(64, activation='relu'),  # Second hidden layer
    Dense(1)  # Output layer
])

# Compile the model
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(X_train, y_train, epochs=100, validation_split=0.2)

model.save('NN2.h5')



In [ ]:
# Load the model
from tensorflow.keras.models import load_model

model = load_model('NN2.h5')

# Desired moment capacity value and tolerance
desired_moment_capacity = 0.5
tolerance = 0.01

# Initial guess for inputs
initial_inputs = tf.Variable(np.random.uniform(low=X.min(axis=0), high=X.max(axis=0), size=(1, X.shape[1])), dtype=tf.float32)

# Loss function: difference between the predicted and desired output
def loss_function():
    predicted_output = model(initial_inputs)
    return tf.reduce_mean(tf.square(predicted_output - desired_moment_capacity))

# Optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Training loop for optimization
for step in range(1000):
    with tf.GradientTape() as tape:
        loss = loss_function()
    grads = tape.gradient(loss, [initial_inputs])
    optimizer.apply_gradients(zip(grads, [initial_inputs]))
    
    if step % 100 == 0:
        current_loss = loss.numpy()
        print(f'Step {step}: Loss = {current_loss}')
        if current_loss < tolerance:
            break

# Admissible inputs
admissible_inputs = initial_inputs.numpy()
print(f'Admissible inputs: {admissible_inputs}')

# Validate admissible inputs by predicting moment capacities
validated_output = model.predict(admissible_inputs)
print(f'Predicted moment capacity: {validated_output[0][0]}')

# Visualize the admissible inputs
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(admissible_inputs[:, 0], admissible_inputs[:, 1], admissible_inputs[:, 2])
ax.set_xlabel('Input 1')
ax.set_ylabel('Input 2')
ax.set_zlabel('Input 3')
plt.show()
